<h1 id="tocheading">XGB Troubleshooting</h1>
<div id="toc"></div>

The purpose of this notebook is to show some comfusion we encoucotered when applying XGboost. We used same data for binary prediction and applied same technique(SMOTE) to handle imbalance. And tried to keep all the hyperparameters the same as well. But still the result is different.  
AUC from SPSS output is .602 vs. .501 in the notebook. I have not checked xgb script in Modeler. Just wondering if  went wrong here.

Unexpected inconsistency:
1. AUC: SPSS(.602) vs. Notebook(.501 or .559)
2. The built-in evaluation produced higher AUC values than roc_auc_score() on test set.

** Any ideas?? **

Note: 
1. Run the following cell to view the full Table of Contents:)
2. I am experiencing trouble creating a new project in DSX right now. If you need to check the datasets(both original and processed data from Modeler), see [here](https://github.com/CatherineCao2016/Datasets). 

In [1]:
import xgboost
xgboost.__version__

'0.7.post3'

In [7]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

## Imports

In [2]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report

## UDFs

In [3]:
def put_file(credentials, local_file_name,outputfilename):  
    f = open(local_file_name,'r')
    my_data = f.read()
    data_to_send = my_data
    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': credentials['username'],'domain': {'id': credentials['domain_id']},
            'password': credentials['password']}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    #print(resp1_body)
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', credentials['container'], '/',  outputfilename])
                            print(url2)
    s_subject_token = resp1.headers['x-subject-token']
    #print(s_subject_token)
    #print(credentials['container'])
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.put(url=url2, headers=headers2, data = data_to_send)
    print(resp2)

In [6]:
# The code was removed by DSX for sharing.

,Dollar_Amount,Transaction_Type,Store_Type,Cardholder_Region,Country,Fraudulent,Last3hourTransactions,Hours_Since_Last_Transaction
0,188.56,swiped,ATM Withdrawal,E,USA,F,4,21
1,160.54,keyed,Grocery,NE,Australia,F,3,21
2,153.24,swiped,Retail - Pharmacy,SW,European Union,T,2,21
3,148.66,keyed,Communications - telephone and wireless,NW,USA,F,1,35
4,174.03,keyed,Spa/Beauty,SW,Asia,F,1,25


## Split into Training and Test

In [7]:
y = df_data_1['Fraudulent'].map(lambda l: 1 if l == 'T' else 0)
X = df_data_1.drop(['Fraudulent'], axis=1)

from sklearn.cross_validation import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234567)

print("Number of training records: " + str(len(X_train)))
print("Number of testing records : " + str(len(X_test)))

Number of training records: 44028
Number of testing records : 11007


/gpfs/fs01/user/s34f-24c5585e483e16-c0ce2641d0ef/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Handle Imbalance using SMOTE(only on Training set)

In [ ]:
# restart kernel if see error
from imblearn.over_sampling import SMOTE

In [ ]:
X_train_dummy = pd.get_dummies(X_train)
sm = SMOTE(random_state=2662761, ratio = 'auto', k_neighbors = 5)
X_train_res, y_train_res = sm.fit_sample(X_train_dummy, y_train)
#smote only deals with continuous variable
X_train_res_rounded = X_train_res.round()


print('Fraud/Non-Fraud Ratio Before SMOTE: ' + str(y_train.sum()/len(y_train)))
print('Fraud/Non-Fraud Ratio After SMOTE: ' + str(y_train_res.sum()/len(y_train_res)))

print("Fraud in Training after SMOTE = T: " + str(sum(y_train_res)))
print("Fraud in Training after SMOTE = F: " + str(len(y_train_res) - sum(y_train_res)))
print("\n************Test Set************\n")
print("Number of testing records : " + str(len(X_test)))
print("Training: T = : " + str(sum(y_test)))
print("Training: F = :" + str(len(y_test) - sum(y_test)))

In [129]:
# save out and retest in spss
py_smoteoutput = pd.DataFrame(X_train_res, columns = pd.get_dummies(X_test).columns.values.tolist())


In [131]:
# The code was removed by DSX for sharing.

In [136]:
py_smoteoutput['Fraud'] = y_train_res

In [140]:
py_smoteoutput['Partition'] = "1_Training"

In [145]:
X_test_dummy = pd.get_dummies(X_test)

In [146]:
X_test_dummy['Fraud'] = y_test

In [147]:
X_test_dummy['Partition'] = "2_Testing"

In [148]:
py_smoteoutput.shape

(83448, 46)

In [149]:
X_test_dummy.shape

(11007, 46)

In [151]:
py_smoteoutput_combined = py_smoteoutput.append(X_test_dummy)

In [152]:
py_smoteoutput_combined.to_csv("py_smoteoutput_combined.csv", index = False)
put_file(credentials_82, "py_smoteoutput_combined.csv", "py_smoteoutput_combined.csv")

https://dal.objectstorage.open.softlayer.com/v1/AUTH_830bc8678c104292bb62a210d196fc7f/Defaultproject/py_smoteoutput_combined.csv
<Response [201]>


In [135]:
X_test_dummy = pd.get_dummies(X_test)
X_test_dummy.to_csv("py_testset.csv", index = False)
put_file(credentials_82, "py_testset.csv", "py_testset.csv")

https://dal.objectstorage.open.softlayer.com/v1/AUTH_830bc8678c104292bb62a210d196fc7f/Defaultproject/py_testset.csv
<Response [201]>


In [103]:
for i in range(len(X_train_res)):
    X_train_res[i][1:] = X_train_res[i][1:].round()

In [74]:
X_train_res[44033]

array([1.33404136e+02, 4.93484248e+00, 1.41303150e+01, 0.00000000e+00,
       1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 9.34842484e-01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.51575156e-02,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       6.51575156e-02, 9.34842484e-01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 9.34842484e-01, 6.51575156e-02])

In [75]:
X_train_res_rounded[44033]

array([133.,   5.,  14.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   1.,   0.])

## XGboost

The hyperparameters are set as following, the same as Modeler's setting.

In [114]:
xgb_model = XGBClassifier(
    tree_method= "auto",
    n_estimators = 10,
    max_depth = 6,
    min_child_weight = 1.0,
    max_delta_step = 0.0,
    objective = "binary:logistic",
    seed = 6924827,
    subsample = 1.0,
    learning_rate = .3,
    gamma = 0.0,
    colsample_bytree = 1.0,
    colsample_bylevel = 1.0,
    reg_lambda = 1.0,
    reg_alpha = 0.0,
    scale_pos_weight = 1.0)

In [115]:
xgb_model.fit(X_train_res, y_train_res, eval_set = [(X_train_res, y_train_res), (pd.get_dummies(X_test).as_matrix(), y_test)], eval_metric = "auc", verbose = True)

[0]	validation_0-auc:0.699632	validation_1-auc:0.65423
[1]	validation_0-auc:0.71747	validation_1-auc:0.652587
[2]	validation_0-auc:0.723259	validation_1-auc:0.649979
[3]	validation_0-auc:0.729915	validation_1-auc:0.655069
[4]	validation_0-auc:0.737506	validation_1-auc:0.657501
[5]	validation_0-auc:0.741308	validation_1-auc:0.655968
[6]	validation_0-auc:0.747141	validation_1-auc:0.655206
[7]	validation_0-auc:0.751209	validation_1-auc:0.656513
[8]	validation_0-auc:0.755906	validation_1-auc:0.655498
[9]	validation_0-auc:0.760441	validation_1-auc:0.657324


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1.0,
       colsample_bytree=1.0, gamma=0.0, learning_rate=0.3,
       max_delta_step=0.0, max_depth=6, min_child_weight=1.0, missing=None,
       n_estimators=10, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0.0,
       reg_lambda=1.0, scale_pos_weight=1.0, seed=6924827, silent=True,
       subsample=1.0, tree_method='auto')

In [116]:
pred_classifier = xgb_model.predict(pd.get_dummies(X_test).as_matrix())

/gpfs/fs01/user/s34f-24c5585e483e16-c0ce2641d0ef/.local/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [118]:
roc = roc_auc_score(y_test, pred_classifier)
print("*****Confusion Matrix *****")
print(confusion_matrix(y_test, pred_classifier))
print("*****Classification Report*****")
print(classification_report(y_test, pred_classifier))
print("ROC: " +  str(roc))

*****Confusion Matrix *****
[[6726 3682]
 [ 244  355]]
*****Classification Report*****
             precision    recall  f1-score   support

          0       0.96      0.65      0.77     10408
          1       0.09      0.59      0.15       599

avg / total       0.92      0.64      0.74     11007

ROC: 0.6194440452252601


In [109]:
if not rounded, roc = .61, if round everything  = .54
*****Confusion Matrix *****
[[8124 2284]
 [ 334  265]]
*****Classification Report*****
             precision    recall  f1-score   support

          0       0.96      0.78      0.86     10408
          1       0.10      0.44      0.17       599

avg / total       0.91      0.76      0.82     11007

ROC: 0.6114787135618036

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 8)

In [139]:
SMOTEoutput

,Dollar_Amount,Transaction_Type,Store_Type,Cardholder_Region,Country,Fraudulent,Last3hourTransactions,Hours_Since_Last_Transaction,Partition
0,188.560000,swiped,ATM Withdrawal,E,USA,F,4,21,1_Training
1,160.540000,keyed,Grocery,NE,Australia,F,3,21,1_Training
2,153.240000,swiped,Retail - Pharmacy,SW,European Union,T,2,21,1_Training
3,148.660000,keyed,Communications - telephone and wireless,NW,USA,F,1,35,2_Testing
4,174.030000,keyed,Spa/Beauty,SW,Asia,F,1,25,1_Training
5,171.950000,swiped,Retail - Home Improvement,SE,USA,F,1,15,1_Training
6,171.720000,keyed,Utilities,NW,USA,F,1,15,1_Training
7,147.310000,swiped,Communications - telephone and wireless,N,Australia,T,1,13,1_Training
8,164.390000,swiped,Grocery,N,USA,F,3,30,1_Training
9,143.110000,keyed,Retail - Apparel,E,USA,F,1,23,1_Training


## Using SMOTE output from SPSS

To control the randomness from Train/Test split and SMOTE, we tried using the SMOTE output from SPSS directly.

In [29]:
SMOTEoutput = pd.read_csv(get_object_storage_file_with_credentials_79f48b7520e04b748bcd13ec11f1f911('Defaultproject', 'modeler_smoteout.csv'))
SMOTEoutput = SMOTEoutput.drop(['Account_Number','Transaction_Time', 'last transaction time', 'Hours Since Last Transaction', 'Transaction_Dates',	'Last_Trasaction_Dates'], 1)


SMOTEoutput.head()

,Dollar_Amount,Transaction_Type,Store_Type,Cardholder_Region,Country,Fraudulent,Last3hourTransactions,Hours_Since_Last_Transaction,Partition
0,188.56,swiped,ATM Withdrawal,E,USA,F,4,21,1_Training
1,160.54,keyed,Grocery,NE,Australia,F,3,21,1_Training
2,153.24,swiped,Retail - Pharmacy,SW,European Union,T,2,21,1_Training
3,148.66,keyed,Communications - telephone and wireless,NW,USA,F,1,35,2_Testing
4,174.03,keyed,Spa/Beauty,SW,Asia,F,1,25,1_Training


In [30]:
train = SMOTEoutput[SMOTEoutput.Partition == "1_Training"].drop('Partition', 1)
test = SMOTEoutput[SMOTEoutput.Partition == "2_Testing"].drop('Partition', 1)

y = train['Fraudulent'].map(lambda l: 1 if l == 'T' else 0)
X = pd.get_dummies(train.drop(['Fraudulent'], axis=1))

y_test_spss = test['Fraudulent'].map(lambda l: 1 if l == 'T' else 0)
X_test_spss = pd.get_dummies(test.drop(['Fraudulent'], axis=1))

In [34]:
xgb_model2 = XGBClassifier(
    tree_method= "auto",
    n_estimators = 10,
    max_depth = 6,
    min_child_weight = 1.0,
    max_delta_step = 0.0,
    objective = "binary:logistic",
    seed = 6924827,
    subsample = 1.0,
    learning_rate = .0001,
    gamma = 0.0,
    colsample_bytree = 1.0,
    colsample_bylevel = 1.0,
    reg_lambda = 1.0,
    reg_alpha = 0.0,
    scale_pos_weight = 2.0)
xgb_model2.fit(X.as_matrix(), y, eval_set = [(X.as_matrix(), y), (X_test_spss.as_matrix(), y_test_spss)], eval_metric = "auc", verbose = True)

[0]	validation_0-auc:0.741674	validation_1-auc:0.596681
[1]	validation_0-auc:0.741682	validation_1-auc:0.596703
[2]	validation_0-auc:0.741715	validation_1-auc:0.596687
[3]	validation_0-auc:0.741686	validation_1-auc:0.596724
[4]	validation_0-auc:0.741683	validation_1-auc:0.596735
[5]	validation_0-auc:0.741686	validation_1-auc:0.596731
[6]	validation_0-auc:0.741685	validation_1-auc:0.596736
[7]	validation_0-auc:0.741686	validation_1-auc:0.59673
[8]	validation_0-auc:0.741685	validation_1-auc:0.596742
[9]	validation_0-auc:0.741686	validation_1-auc:0.59673


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1.0,
       colsample_bytree=1.0, gamma=0.0, learning_rate=0.0001,
       max_delta_step=0.0, max_depth=6, min_child_weight=1.0, missing=None,
       n_estimators=10, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0.0,
       reg_lambda=1.0, scale_pos_weight=2.0, seed=6924827, silent=True,
       subsample=1.0, tree_method='auto')

In [35]:
pred_classifier_spssoutput = xgb_model2.predict(X_test_spss.as_matrix())

/gpfs/fs01/user/s34f-24c5585e483e16-c0ce2641d0ef/.local/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [36]:
roc2 = roc_auc_score(y_test_spss, pred_classifier_spssoutput)
print("ROC: " +  str(roc2))

ROC: 0.5753863369337515


Still the ROC is different. 

## Appendix

Some screenshots from SPSS:
<img src="https://github.com/CatherineCao2016/pics/raw/master/xgb_modeler_1.png" width="500" height="200" align="middle"/>
<img src="https://github.com/CatherineCao2016/pics/raw/master/xgb_modeler_2.png" width="400" height="200" align="middle"/>
<img src="https://github.com/CatherineCao2016/pics/raw/master/xgb_modeler_3.png" width="400" height="200" align="middle"/>
<img src="https://github.com/CatherineCao2016/pics/raw/master/xgb_modeler_4.png" width="400" height="200" align="middle"/>
<img src="https://github.com/CatherineCao2016/pics/raw/master/xgb_modeler_5.png" width="400" height="200" align="middle"/>